### Benson Team Awesome

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


|Code      | explanation                                                                                     |
|:-------- |:----------------------------------------------------------------------------------------------- | 
| C/A      | Control Area (A002)                                                                          | 
| UNIT     | Remote Unit for a station (R051)                                                             | 
| SCP      | Subunit Channel Position represents an specific address for a device (02-00-00)              | 
| STATION  | Represents the station name the device is located at                                         | 
| LINENAME | Represents all train lines that can be boarded at this station                               | 
|          |   Normally lines are represented by one character.  LINENAME 456NQR repersents train server for | 
|          |   4, 5, 6, N, Q, and R trains.                                                                  | 
| DIVISION | Represents the Line originally the station belonged to BMT, IRT, or IND                      | 
| DATE     | Represents the date (MM-DD-YY)                                                               |   
| TIME     | Represents the time (hh:mm:ss) for a scheduled audit event                                   | 
| DESc     | Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)                | 
|          |   1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.         | 
|          |   2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. |
|ENTRIES   | The comulative entry register value for a device|
|EXIST     | The cumulative exit register value for a device|


In [ ]:
import csv
import urllib
import io
import collections as clt
import pandas as pd
import numpy as np
import datetime as dt
import multiprocessing as mlp
import matplotlib.pyplot as plt
#data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt')
# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [ ]:
list_data =[]
list_weeks = ['160102', '160109', '160116','160130','160206','160213',
              '160220','160227','160305','160312','160319','160326','160402']
for u in list_weeks:
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + u + '.txt'
    print(url)
    list_data.append(pd.read_csv(url))

data = pd.concat(list_data)
del list_data

In [ ]:
data.head()

In [ ]:
data.columns = data.columns.str.strip()
data["datetime"] = data["DATE"] + ' ' + data["TIME"]
data.datetime = pd.to_datetime(data.datetime)

In [ ]:
df.sort_values(['STATION','datetime'], ascending=[True, True])

data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS
data['dTOTAL'] = data.dENTRIES+ data.dEXITS

mask = ((data.dENTRIES >= 0) & (data.dENTRIES < 5000)) | ((data.dEXITS >= 0) & (data.dEXITS < 5000))
df = data.ix[mask,:]
del mask

In [ ]:
df.head()

In [ ]:
#print(data.head())
#print(data['STATION'].unique())
df = df.groupby(['STATION', 'datetime']).sum()
df.reset_index()
idx = pd.IndexSlice
dt_station = df.loc[idx['A002', 'R051', :, '59 ST'], :]
df.reset_index(inplace=True)
df.sort_values(['STATION','datetime'], ascending=[True, True])
df.shape

In [ ]:
idx = pd.IndexSlice
st_59_val = df.loc[idx['A002', 'R051', :, '59 ST'], :]

st_59_val

In [ ]:
df.reset_index()

In [ ]:
list(data.columns.values)

In [ ]:
mask = (data.TIME == '00:00:00') | (data.TIME=='01:00:00')

In [ ]:
datatest = data.ix[mask, :]

In [ ]:
mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)
datatest2 = datatest.ix [mask2, :]

In [ ]:
datatest2 = datatest.ix [mask2, :]

In [ ]:
final = datatest2.groupby([datatest2.DATE,datatest2.STATION]).sum()
df = final.add_suffix('_Count').reset_index()
mask3 = (datatest2.dEXITS >= 0) & (datatest2.dEXITS <=5000)
#Changing to data frame

In [ ]:
df.sort(['dTOTAL_Count'], ascending=False, inplace=True)


In [ ]:
final = datatest2.groupby([datatest2.DATE,datatest2.STATION]).sum()
mask3 = (datatest2.dEXITS >= 0) & (datatest2.dEXITS <=5000)
df = final.add_suffix('_Count').reset_index()
df.sort(['dTOTAL_Count'], ascending=False, inplace=True)

In [ ]:
#df_night = data[data['TIME' ==  ]
#data.head()
raw_readings ={}
raw_readings.setdefault(tuple(['AA','BB']), pd.DataFrame()).append(pd.DataFrame(['5','6']))
raw_readings.keys()

In [ ]:
#Create the keys that is used to make a mask
data_dict = clt.OrderedDict()
data['id'] = data.iloc[:, 0]
for j in range(data.shape[0]):
    data_dict.setdefault(tuple(data.ix[j,0:4]), pd.DataFrame())
        
#drop the keys that is 
#mask_key = data['id'].drop_duplicates()
#mask_ind = np.arange(mask_key.shape[0])
#data.id.replace(mask_key.as_matrix(), mask_ind, inplace=True)
#test = data.sort_values(['Delta'])
#data.drop(['C/A', 'UNIT', 'SCP','STATION'], axis=1, inplace=True)
#data.drop(['LINENAME', 'DIVISION', 'DATE','TIME'], axis=1, inplace=True)
#print(mask_key.as_matrix())
data_dict

In [ ]:
#Initialize the Delta columns as zeros
data['Delta_ENTRIES'] = np.zeros(data.shape[0])
data['Delta_EXITS'] = np.zeros(data.shape[0])

#temporary function to fill in the deltas by selections
#a0 = data['C/A'].drop_duplicates()
#a1 = data['UNIT'].drop_duplicates()
#a2 = data['SCP'].drop_duplicates()
#a3 = data['STATION'].drop_duplicates()
#for i0 in a0:
#    for i1 in a1:
        for 
def fill_in(mask):
    data.ix[mask,'Delta_ENTRIES'] = (data.loc[mask, 'ENTRIES'].shift(1) - data.loc[mask, 'ENTRIES'].shift(-1))
    data.ix[mask,'Delta_EXITS'] = (data.loc[mask, 'EXITS'].shift(1) - data.loc[mask, 'EXITS'].shift(-1))

In [ ]:
for s in mask_ind:
    mask = data['id'].isin(s)
    fill_in(mask)

In [ ]:
data['Delta_ENTRIES'] = (data['ENTRIES'] - data['ENTRIES'].shift(-1))
data['Delta_EXITS'] = (data['EXITS'] - data['EXITS'].shift(-1))
#print(data.head(20))
print(data.head(50))


In [ ]:
df.mask('DIVISION', 'BMT')

In [ ]:
import gmaps
data = gmaps.datasets.load_dataset('taxi_rides')
gmaps.heatmap(data)

In [6]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.plotly as py
A = pd.DataFrame(np.random.randn(4, 4), columns=["A", "B", "C", "D"])
#A = A.mask("A", 0.7).mask("B", 0.2)


py.sign_in('ynot79', '2r4xdvecbq')

##This is the login info you need to use this module

import cufflinks as cf

import pandas as pd

## if you get an error... 

##pip install cufflinks --upgrade

##pip install plotly --upgrade

cf.set_config_file(offline=False, world_readable=True, theme='pearl')

df.iplot(kind='bubble', x='DATE', y='dTOTAL', size='dTOTAL', text='STATION', xTitle='test', yTitle='test', filename='cufflinks/simple-bubble-chart')

#if you go to plotty... all sorts of plots we can use.



NameError: name 'df' is not defined